## Match searches with bookings

- For every search in the searches file, find out whether the search ended up in a booking or not (using the info in the bookings file). For instance, search and booking origin and destination should match. 

- For the bookings file, origin and destination are the columns dep_port and arr_port, respectively. 

- Generate a CSV file with the search data, and an additional field, containing 1 if the search ended up in a booking, and 0 otherwise.



## 2) Prepare the data for processing

### Booking

#### We didnt check for duplicates so far... What if the file is has duplicated lines?

In [ ]:
import pandas as pd
path='/home/dsc/Data/challenge'
#file_name='bookings.sample.csv.bz2'
file_name='bookings.csv.bz2'
filepath=path+'/'+file_name



bc=pd.read_csv(filepath,sep='^', chunksize=300000, low_memory=False)

all_chunks=pd.DataFrame()
for i,chunk in enumerate(bc):
    all_chunks=all_chunks.append(chunk)
    all_chunks=all_chunks.drop_duplicates()
    print (i, len(all_chunks))

#### We have seen that we have white space in some columns....

In [11]:
all_chunks.columns.tolist()
#list(all_chunks.columns)

['act_date           ',
 'source',
 'pos_ctry',
 'pos_iata',
 'pos_oid  ',
 'rloc          ',
 'cre_date           ',
 'duration',
 'distance',
 'dep_port',
 'dep_city',
 'dep_ctry',
 'arr_port',
 'arr_city',
 'arr_ctry',
 'lst_port',
 'lst_city',
 'lst_ctry',
 'brd_port',
 'brd_city',
 'brd_ctry',
 'off_port',
 'off_city',
 'off_ctry',
 'mkt_port',
 'mkt_city',
 'mkt_ctry',
 'intl',
 'route          ',
 'carrier',
 'bkg_class',
 'cab_class',
 'brd_time           ',
 'off_time           ',
 'pax',
 'year',
 'month',
 'oid      ']

In [12]:
all_chunks.columns=all_chunks.columns.map(lambda x:str(x).strip())

In [26]:
all_chunks.columns.tolist()

['act_date',
 'source',
 'pos_ctry',
 'pos_iata',
 'pos_oid',
 'rloc',
 'cre_date',
 'duration',
 'distance',
 'dep_port',
 'dep_city',
 'dep_ctry',
 'arr_port',
 'arr_city',
 'arr_ctry',
 'lst_port',
 'lst_city',
 'lst_ctry',
 'brd_port',
 'brd_city',
 'brd_ctry',
 'off_port',
 'off_city',
 'off_ctry',
 'mkt_port',
 'mkt_city',
 'mkt_ctry',
 'intl',
 'route',
 'carrier',
 'bkg_class',
 'cab_class',
 'brd_time',
 'off_time',
 'pax',
 'year',
 'month',
 'oid']

In [13]:
b=all_chunks[['cre_date','act_date', 'dep_port', 'arr_port']].copy()
#si es un round trip lo vamos a tener 2 veces

In [17]:
#b.head()
b.iloc[0].tolist()

['2013-02-22 00:00:00', '2013-03-05 00:00:00', 'ZRH     ', 'LHR     ']

In [20]:
b.dep_port=b.dep_port.map(lambda x:str(x).strip())
b.arr_port=b.arr_port.map(lambda x:str(x).strip())
##b['arr_port']=b['arr_port'].map(lambda x: x.strip())

In [21]:
#b.head()
b.iloc[0].tolist()

['2013-02-22 00:00:00', '2013-03-05 00:00:00', 'ZRH', 'LHR']

#### Could we do this with command line?

###  Search

In [ ]:
%%time
#12 min
import pandas as pd
path='/home/dsc/Data/challenge'
#file_name='searches.sample.csv.bz2'
file_name='searches.csv.bz2'
filepath=path+'/'+file_name


all_chunks=pd.DataFrame()
bc=pd.read_csv(filepath,sep='^', chvunksize=300000, low_memory=False)

for i,chunk in enumerate(bc):
    all_chunks=all_chunks.append(chunk)
    all_chunks=all_chunks.drop_duplicates()
    print (i, len(all_chunks))
    
    
    

In [28]:
s=all_chunks[['Date', 'Origin','Destination']]

In [29]:
s.head()

,Date,Origin,Destination
0,2013-01-01,TXL,AUH
1,2013-01-01,ATH,MIL
2,2013-01-01,ICT,SFO
3,2013-01-01,RNB,ARN
4,2013-01-01,OSL,MAD


### NaN and prepare dates

In [35]:
print "size of Book:\t%d \nsize of Search:\t%d " % (len(b), len(s))

size of Book:	1299993 
size of Search:	718003 


In [36]:
b=b.dropna()

In [37]:
b['act_date']=pd.to_datetime(b['act_date'],errors='raise')

In [38]:
b.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1299993 entries, 0 to 299999
Data columns (total 4 columns):
cre_date    1299993 non-null object
act_date    1299993 non-null datetime64[ns]
dep_port    1299993 non-null object
arr_port    1299993 non-null object
dtypes: datetime64[ns](1), object(3)
memory usage: 49.6+ MB


In [39]:
b.drop('cre_date', axis=1, inplace=True)
b.head()

,act_date,dep_port,arr_port
0,2013-03-05,ZRH,LHR
1,2013-03-26,SAL,CLT
2,2013-03-26,SAL,CLT
3,2013-03-26,AKL,SVO
4,2013-03-26,AKL,SVO


In [42]:
s=s.dropna()
s['Date'] = pd.to_datetime(s['Date'],errors='raise')

In [43]:
print "size of Book:\t%d \nsize of Search:\t%d " % (len(b), len(s))

size of Book:	1299993 
size of Search:	718002 


## 3) Make action plan

In [2]:
import pandas as pd

path='/home/dsc/Data/challenge'
file_nameb='bookings_wn_dup.csv.bz2'
file_names='searches_wn_dup.csv.bz2'

filepathb=path+'/'+file_nameb
filepaths=path+'/'+file_names

b=pd.read_csv(file_nameb,sep='^',usecols=['act_date           ','dep_port', 'arr_port'])
s=pd.read_csv(file_names,sep='^',usecols=['Date', 'Origin','Destination'])
print len(s)
#s.tail()

718003


In [3]:
b=b.dropna()
b.columns=b.columns.map(lambda x:str(x).strip())
b.dep_port=b.dep_port.map(lambda x:str(x).strip())
b.arr_port=b.arr_port.map(lambda x:str(x).strip())
b['act_date']=pd.to_datetime(b['act_date'],errors='raise')

s.reset_index(inplace=True) #we should save the index if we are to merge 1/0 to these values, and if there are some NaN

s=s.dropna()
s['Date'] = pd.to_datetime(s['Date'],errors='raise')

In [17]:
#s.tail(10)

In [4]:
print "size of Book:\t%d \nsize of Search:\t%d " % (len(b), len(s))

size of Book:	1299993 
size of Search:	718002 


In [6]:
b.iloc[0].tolist()

[Timestamp('2013-03-05 00:00:00'), 'ZRH', 'LHR']

In [29]:
#b_no_dup=b.drop_duplicates()

In [5]:
b_gr=b.groupby(['dep_port','arr_port', 'act_date'])

In [20]:
for i, (group, values) in enumerate(b_gr):
    print group
    print values
    if (i==2) :break

('AAB', 'ROV', Timestamp('2013-05-29 00:00:00'))
         act_date dep_port arr_port
387078 2013-05-29      AAB      ROV
387079 2013-05-29      AAB      ROV
('AAB', 'ROV', Timestamp('2013-08-24 00:00:00'))
         act_date dep_port arr_port
572522 2013-08-24      AAB      ROV
572523 2013-08-24      AAB      ROV
('AAE', 'ALG', Timestamp('2013-01-07 00:00:00'))
         act_date dep_port arr_port
165186 2013-01-07      AAE      ALG


In [6]:
b_gr=b.groupby(['dep_port','arr_port', 'act_date']).size().reset_index()
b_gr.head()

,dep_port,arr_port,act_date,0
0,AAB,ROV,2013-05-29,2
1,AAB,ROV,2013-08-24,2
2,AAE,ALG,2013-01-07,1
3,AAE,ALG,2013-01-08,2
4,AAE,ALG,2013-01-09,6


In [7]:
b_gr.rename(columns={0:'Num of Bookings'}, inplace=True)
print len(b_gr)
b_gr.head()

466228


,dep_port,arr_port,act_date,Num of Bookings
0,AAB,ROV,2013-05-29,2
1,AAB,ROV,2013-08-24,2
2,AAE,ALG,2013-01-07,1
3,AAE,ALG,2013-01-08,2
4,AAE,ALG,2013-01-09,6


In [8]:
s.head()

,index,Date,Origin,Destination
0,0,2013-01-01,TXL,AUH
1,1,2013-01-01,ATH,MIL
2,2,2013-01-01,ICT,SFO
3,3,2013-01-01,RNB,ARN
4,4,2013-01-01,OSL,MAD


In [130]:
pd.merge?

In [8]:
s_b=s.merge(b_gr, 
            how='left', 
            left_on=['Date','Origin','Destination'], 
            right_on=['act_date', 'dep_port','arr_port'])

s_b.drop(['dep_port','arr_port', 'act_date'],axis=1, inplace=True)
s_b[s_b['Num of Bookings']>0].head()

,index,Date,Origin,Destination,Num of Bookings
23,23,2013-01-01,DUS,IST,1
27,27,2013-01-01,RUH,JED,17
40,40,2013-01-01,DMM,MNL,1
59,59,2013-01-01,ATL,MIA,4
94,94,2013-01-01,DXB,KUL,1


#### If we dont want to drop the duplicates

In [52]:
b_gr2=b_gr.rename(columns={0:'Num of Bookings', 'act_date':'Date', 'dep_port':'Origin','arr_port':'Destination'})
s_b=s.merge(b_gr2, 
            how='left')
s_b[s_b['Num of Bookings']>0].head()

,Date,Origin,Destination,Num of Bookings
23,2013-01-01,DUS,IST,1
27,2013-01-01,RUH,JED,17
40,2013-01-01,DMM,MNL,1
59,2013-01-01,ATL,MIA,4
94,2013-01-01,DXB,KUL,1


In [10]:
len(s_b), len(s)

(718002, 718002)

### Now we have to put 1 to all the booking with Num of Bookings>1 in the clean file

In [9]:
s_all=pd.read_csv(file_names,sep='^', low_memory=False)
s_all.head()

,Date,Time,TxnCode,OfficeID,Country,Origin,Destination,RoundTrip,NbSegments,Seg1Departure,...,Seg6Arrival,Seg6Date,Seg6Carrier,Seg6BookingCode,From,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice
0,2013-01-01,20:25:57,MPT,624d8c3ac0b3a7ca03e3c167e0f48327,DE,TXL,AUH,1,2,TXL,...,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,FRA
1,2013-01-01,10:15:33,MPT,b0af35b31588dc4ab06d5cf2986e8e02,MD,ATH,MIL,0,1,ATH,...,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,KIV
2,2013-01-01,18:04:49,MPT,3561a60621de06ab1badc8ca55699ef3,US,ICT,SFO,1,2,ICT,...,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,NYC
3,2013-01-01,17:42:40,FXP,1864e5e8013d9414150e91d26b6a558b,SE,RNB,ARN,0,1,RNB,...,NaN,NaN,NaN,NaN,1ASI,0,0,0,d41d8cd98f00b204e9800998ecf8427e,STO
4,2013-01-01,17:48:29,MPT,1ec336348f44207d2e0027dc3a68c118,NO,OSL,MAD,1,2,OSL,...,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,OSL


In [25]:
# s_all=s_all.dropna(subset = ['Date', 'Origin','Destination'])

In [10]:
len(s_all),len(s_b)

(718003, 718002)

In [12]:
result=s_all.merge(s_b, 
            how='left', 
            left_index=True,
            right_on='index')

In [14]:
result[result['Num of Bookings']>1].tail(10)

,Date_x,Time,TxnCode,OfficeID,Country,Origin_x,Destination_x,RoundTrip,NbSegments,Seg1Departure,...,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice,index,Date_y,Origin_y,Destination_y,Num of Bookings
717831,2013-05-03,01:33:05,MPT,8b4fa4e121637f583bfa86ef0fa26c9e,US,LAX,LAS,1,2,LAX,...,0,0,0,d41d8cd98f00b204e9800998ecf8427e,LAS,717832,2013-05-03,LAX,LAS,2
717856,2013-05-03,13:59:20,MPT,440642a9bdaeb6287f826cefd73255e8,US,KCH,KUL,1,2,KCH,...,0,0,0,d41d8cd98f00b204e9800998ecf8427e,HPN,717857,2013-05-03,KCH,KUL,2
717859,2013-05-03,21:04:28,MPT,f976842f00052a55080c5ea015cd728c,US,JFK,SDQ,1,2,JFK,...,0,0,0,d41d8cd98f00b204e9800998ecf8427e,SFO,717860,2013-05-03,JFK,SDQ,2
717885,2013-05-03,11:45:21,MPT,cf4ae41b25a075c3a8fee145e1919199,NL,AMS,CDG,1,2,AMS,...,0,0,0,d41d8cd98f00b204e9800998ecf8427e,AMS,717886,2013-05-03,AMS,CDG,4
717892,2013-05-03,17:17:18,FXX,1722290df1e162d326280c5d4a82657b,CO,BAQ,BOG,0,1,BAQ,...,0,0,0,c01f2026c4c0064e4a406973f52e1375,BOG,717893,2013-05-03,BAQ,BOG,4
717914,2013-05-03,16:00:14,FXL,9c7e0c8dd5b434e375f293130d54aa98,CO,MDE,BOG,1,2,MDE,...,0,0,0,40b17e34908fcd5f252369ea4aa546c7,CLO,717915,2013-05-03,MDE,BOG,8
717926,2013-05-03,14:34:47,MPT,d1b80a3dca253a3816073469f2cfe6be,GB,LHR,MIA,1,2,LHR,...,0,0,0,d41d8cd98f00b204e9800998ecf8427e,LON,717927,2013-05-03,LHR,MIA,2
717943,2013-05-03,15:46:07,MTC,7f756d89441bf77ca6cfc2c69e358aa8,ES,MAD,SFO,0,1,MAD,...,0,0,0,d41d8cd98f00b204e9800998ecf8427e,MAD,717944,2013-05-03,MAD,SFO,4
717964,2013-05-03,04:37:54,AWT,ccb20b39ccfeccfe2cf6ee59fe243093,FR,CDG,LAX,0,1,CDG,...,0,0,0,d41d8cd98f00b204e9800998ecf8427e,CDG,717965,2013-05-03,CDG,LAX,2
717966,2013-05-03,18:59:04,CAL,c500a9ba3f7a21c286fdefc5c8c2ff99,BG,SOF,PMI,1,2,SOF,...,0,0,0,d41d8cd98f00b204e9800998ecf8427e,SOF,717967,2013-05-03,SOF,PMI,2


In [23]:
result.drop(['Date_y','Origin_y','Destination_y'], axis=1, inplace=True)
result.rename(columns={'Date_x':'Date','Num of Bookings':'Booked'}, inplace=True)
result['Booked']=result['Booked'].fillna(0).map(lambda x: 1 if x>0 else 0)

In [29]:
result.to_csv('searches_bookings.csv',sep='^', index=False)

,Date,Time,TxnCode,OfficeID,Country,Origin_x,Destination_x,RoundTrip,NbSegments,Seg1Departure,...,Seg6Carrier,Seg6BookingCode,From,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice,index,Booked
0,2013-01-01,20:25:57,MPT,624d8c3ac0b3a7ca03e3c167e0f48327,DE,TXL,AUH,1,2,TXL,...,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,FRA,0,0
1,2013-01-01,10:15:33,MPT,b0af35b31588dc4ab06d5cf2986e8e02,MD,ATH,MIL,0,1,ATH,...,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,KIV,1,0
2,2013-01-01,18:04:49,MPT,3561a60621de06ab1badc8ca55699ef3,US,ICT,SFO,1,2,ICT,...,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,NYC,2,0
3,2013-01-01,17:42:40,FXP,1864e5e8013d9414150e91d26b6a558b,SE,RNB,ARN,0,1,RNB,...,NaN,NaN,1ASI,0,0,0,d41d8cd98f00b204e9800998ecf8427e,STO,3,0
4,2013-01-01,17:48:29,MPT,1ec336348f44207d2e0027dc3a68c118,NO,OSL,MAD,1,2,OSL,...,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,OSL,4,0


### Bonus: Check if all airports are letters

In [ ]:
int(s.ix[317322]['Origin'])

In [204]:
def check_airports (airport):
    try:
        int(airport)
        print "%s is not airport"%(airport)
        return False
    except:
        return True

In [44]:
b=b.reset_index()
b.head()

,index,act_date,dep_port,arr_port
0,0,2013-03-05,ZRH,LHR
1,1,2013-03-26,SAL,CLT
2,2,2013-03-26,SAL,CLT
3,3,2013-03-26,AKL,SVO
4,4,2013-03-26,AKL,SVO


In [199]:
check_airports('123')

123 is not airport


False

In [205]:
s.head().Origin.map(check_airports)

0    True
1    True
2    True
3    True
4    True
Name: Origin, dtype: bool

In [206]:
s_test=s.head()

s_test2 = s[s.Origin.map(check_airports)]
s_test2

240 is not airport


,Date,Origin,Destination
0,2013-01-01,TXL,AUH
1,2013-01-01,ATH,MIL
2,2013-01-01,ICT,SFO
3,2013-01-01,RNB,ARN
4,2013-01-01,OSL,MAD
5,2013-01-01,IAH,BLR
6,2013-01-01,CPH,PAR
7,2013-01-01,PAR,DUB
8,2013-01-01,DUS,ACE
9,2013-01-01,FRA,BGW


In [106]:
s = s[s.Origin.map(check_airports)]
s = s[s.Destination.map(check_airports)]
len(s)

422182